In [1]:
import torch as th
from active_critic.utils.pytorch_utils import calcMSE

In [3]:
def tokenize_input(inpt:th.Tensor, ntokens:int, minimum:float, maximum:float):
    output = th.zeros([inpt.shape[0], inpt.shape[1], ntokens*inpt.shape[2]])
    

In [4]:
batch_size = 2
seq_len = 3
dim = 4

a = th.rand([batch_size, seq_len, dim])
a[0,0,0] = -1
a[0,0,1] = 1
minimum = -1
maximum = 1
ntokens = 10
scale = maximum - minimum
trans_a = ((a - minimum) / scale)*(ntokens-1)

In [5]:
def make_indices(num, repeats):
    indices = th.arange(num).reshape([1,-1]).repeat([repeats,1]).T.reshape([-1])
    return indices

In [6]:
def tokenize(inpt, minimum, maximum, ntokens):
    scale = maximum - minimum
    rec_inpt = ((inpt - minimum) / scale)*(ntokens-1)
    rounded = th.round(rec_inpt).type(th.long).reshape([-1])
    batch_indices = make_indices(batch_size, seq_len*dim).to(inpt.device)
    seq_indices = make_indices(seq_len, dim).repeat(batch_size).to(inpt.device)
    dim_indices = make_indices(dim, 1).repeat(batch_size*seq_len).to(inpt.device)
    result = th.zeros([inpt.shape[0], inpt.shape[1], inpt.shape[2], ntokens], dtype=float, device=inpt.device)
    result[tuple((batch_indices, seq_indices, dim_indices, rounded))] = 1
    result = result.reshape([inpt.shape[0], inpt.shape[1], -1])
    return result
    
    

In [7]:
new_result = tokenize(inpt=a, minimum=-1, maximum=1, ntokens=ntokens)

In [10]:
new_result.shape

torch.Size([2, 3, 40])

In [9]:
new_result.argmax(dim=-1)

tensor([[0, 5, 6],
        [7, 5, 6]])

In [11]:
def detokenize(inpt:th.Tensor, minimum, maximum, ntokens):
    exp_inpt = inpt.reshape([inpt.shape[0], inpt.shape[0], -1, ntokens])
    values = exp_inpt.argmax(dim=-1)
    scale = maximum - minimum
    values = values/(ntokens-1) * scale
    values = values.reshape([inpt.shape[0], inpt.shape[1], -1])
    values = values + minimum
    return values

In [12]:
new_a = detokenize(new_result, minimum=-1, maximum=1, ntokens=ntokens)

In [13]:
def unstack_tokens(inpt, ntokens):
    result = inpt.reshape([inpt.shape[0], inpt.shape[1], -1, ntokens])
    return result

In [14]:
unstacked_result = unstack_tokens(new_result, ntokens=ntokens)

In [15]:
th.abs(a - new_a) < (maximum - minimum)/ntokens

tensor([[[True, True, True, True],
         [True, True, True, True],
         [True, True, True, True]],

        [[True, True, True, True],
         [True, True, True, True],
         [True, True, True, True]]])